![logos](https://cdn-images-1.medium.com/max/800/1*T8Pnw0kiVbrPGnqnB2I_Zw.jpeg)

**k-Nearest-Neighbours (k-NN)** é um modelo supervisionado de aprendizado de máquina. A aprendizagem supervisionada é quando um modelo aprende com dados já rotulados. Um modelo de **aprendizado supervisionado** recebe um conjunto de objetos de entrada e valores de saída. Em seguida, o modelo treina esses dados para aprender a mapear as entradas para a saída desejada para aprender a fazer previsões sobre dados não vistos.

Os modelos k-NN funcionam tomando um ponto de dados e observando os pontos de dados rotulados mais próximos de 'k'. O ponto de dados é então atribuído ao rótulo da maioria dos pontos mais próximos 'k'.

In [1]:
import pandas as pd

# carga do csv via pandas
df = pd.read_csv('diabetes_data.csv')

# visualizar as 5 primeiras linhas
df.head()

,pregnancies,glucose,diastolic,triceps,insulin,bmi,dpf,age,diabetes
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


In [2]:
# numeros de linhas e colunas do dataframe
df.shape

(768, 9)

In [3]:
# lista com as colunas numéricas que não são necessariamente contínus
numerical_columns = [col for col in df.columns if (df[col].dtype=='int64' or df[col].dtype=='float64')]

In [4]:
numerical_columns

['pregnancies',
 'glucose',
 'diastolic',
 'triceps',
 'insulin',
 'bmi',
 'dpf',
 'age',
 'diabetes']

In [5]:
df.dtypes

pregnancies      int64
glucose          int64
diastolic        int64
triceps          int64
insulin          int64
bmi            float64
dpf            float64
age              int64
diabetes         int64
dtype: object

In [6]:
# criação de um df sem a coluna-alvo
X = df.drop(columns=['diabetes'])
X.head()

,pregnancies,glucose,diastolic,triceps,insulin,bmi,dpf,age
0,6,148,72,35,0,33.6,0.627,50
1,1,85,66,29,0,26.6,0.351,31
2,8,183,64,0,0,23.3,0.672,32
3,1,89,66,23,94,28.1,0.167,21
4,0,137,40,35,168,43.1,2.288,33


In [7]:
# separando os valores alvos usando conceito de array
y = df['diabetes'].values

#view target values
y[0:5]

array([1, 0, 1, 0, 1])

In [8]:
# Agora vamos dividir o conjunto de dados em dados de treinamento e dados de teste. Os dados de treinamento 
# são os dados que o modelo aprenderá. Os dados de teste são os dados que usaremos para ver o desempenho do 
# modelo em dados não vistos.
# O Scikit-learn tem uma função que podemos usar chamada 'train_test_split' que facilita a divisão de nosso 
# conjunto de dados em dados de treinamento e teste.
from sklearn.model_selection import train_test_split

#split dataset into train and test data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state=1, stratify=y)

In [9]:
# 'train_test_split' aceita 5 parâmetros. Os dois primeiros parâmetros são os dados de entrada e destino que 
# dividimos anteriormente. Em seguida, definiremos 'test_size' para 0,2. Isso significa que 20% de todos os 
# dados serão usados para teste, o que deixa 80% dos dados como dados de treinamento para o modelo aprender. 
# Definir 'random_state' como 1 garante que recebamos a mesma divisão a cada vez para que possamos reproduzir 
# nossos resultados.
from sklearn.neighbors import KNeighborsClassifier

# criação do classificador KNN
knn = KNeighborsClassifier(n_neighbors = 3)

# Fit the classifier to the data
knn.fit(X_train,y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=3, p=2,
           weights='uniform')

In [10]:
# mostra primeiro 5 previsões de modelo nos dados de teste 
knn.predict(X_test)[0:5]

array([0, 0, 0, 0, 1])

In [11]:
# checa a acurácia dos dados testes
knn.score(X_test, y_test)

0.6688311688311688

Nosso modelo tem uma precisão de aproximadamente **66,88%**

Validação cruzada

![logos](https://cdn-images-1.medium.com/max/800/1*NyvaFiG_jXcGgOaouumYJQ.jpeg)

In [12]:
from sklearn.model_selection import cross_val_score
import numpy as np

#create a new KNN model
knn_cv = KNeighborsClassifier(n_neighbors=3)

#train model with cv of 5 
cv_scores = cross_val_score(knn_cv, X, y, cv=5)

# média dos 5 melhores valores 
print(cv_scores)
print('cv_scores mean:{}'.format(np.mean(cv_scores)))

[0.68181818 0.69480519 0.75324675 0.75163399 0.68627451]
cv_scores mean:0.7135557253204311


Usando a validação cruzada, nossa pontuação média é de cerca de **71,36%**

In [13]:
from sklearn.model_selection import GridSearchCV

# novo modelo KNN
knn2 = KNeighborsClassifier()

# dicionário com todos oa valores para teste 
param_grid = {'n_neighbors': np.arange(1, 25)}
knn_gscv = GridSearchCV(knn2, param_grid, cv=5)

knn_gscv.fit(X, y)

GridSearchCV(cv=5, error_score='raise',
       estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform'),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'n_neighbors': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [14]:
# checar o melhor número de vizinhos KNN's
knn_gscv.best_params_

{'n_neighbors': 14}

In [15]:
# calcula o melhor valor de KNN's
knn_gscv.best_score_

0.7578125